# 🚆Train_seoul
서울을 지나는 지하철 노선만 추리기

### TODO
- [X] 노선별로 지역 경계 확인
- [X] 경계에서 벗어나는 노선 제거
- [ ] 경계에서 벗어나는 역 제거

### 0. import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import set_matplotlib_formats
import koreanize_matplotlib
import folium
from folium.plugins import MarkerCluster
import json

### 1. 노선별로 지역 경계 확인

In [6]:
# 데이터 읽어오기
station = pd.read_csv('result_train_station.csv')

routes = ['1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '8호선', '9호선', '경강선', '경의중앙선', '경춘선',
          '공항철도선', '김포골드라인', '서해선', '수인분당선', '신림선', '신분당선', '에버라인선', '우이신설선', '의정부선', '인천1호선', '인천2호선']

# 파일로 저장
for i in range(len(routes)):
    geo_station = station[station["노선명"] == routes[i]].copy()
    
    # 중심 설정
    fmap = folium.Map(location=[geo_station['위도'].mean(
    ), geo_station['경도'].mean()], zoom_start=8)

    # 행정구역 표시
    geo_line = json.load(open('../TL_SCCO_CTPRVN.json', encoding='utf-8'))
    folium.GeoJson(geo_line).add_to(fmap)

    # 마커 표시
    for n in geo_station.index:
        # 팝업에 들어갈 텍스트를 지정
        popup_name = str(geo_station.loc[n, '위도']) + ", " + str(geo_station.loc[n, '경도']) + \
            " \n" + geo_station.loc[n, '역사명'] + ' - ' + geo_station.loc[n, '노선명']
        
        folium.Marker(
            location=[geo_station.loc[n, '위도'], geo_station.loc[n, '경도']],
            popup=popup_name, 
            tooltip=popup_name
        ).add_to(fmap)
    
    fmap.save(routes[i] + '_station.html')
    
# 확인 결과, 경강선, 김포골드라인, 서해선, 에버라인선, 의정부선, 인천1/2호선은 아예 서울을 지나지 않거나 한 역만 지남
# 또한, 서울 -> 서울 환승 구간에도 사용되지 않을 것 같음 (멀어지는 방향이므로)
# 따라서, 해당 노선들에 대한 역은 제거해도 된다고 판단
# 다른 노선들에 대한 역정보는 일단 제거하지 않거나 추후 허브를 만들 때 제거하기


### 2. 경계에서 벗어나는 노선 제거

In [8]:
station = station[(station['노선명'] != '경강선')]
station = station[(station['노선명'] != '김포골드라인')]
station = station[(station['노선명'] != '서해선')]
station = station[(station['노선명'] != '에버라인선')]
station = station[(station['노선명'] != '의정부선')]
station = station[(station['노선명'] != '인천1호선')]
station = station[(station['노선명'] != '인천2호선')]

set(station['노선명'])

{'1호선',
 '2호선',
 '3호선',
 '4호선',
 '5호선',
 '6호선',
 '7호선',
 '8호선',
 '9호선',
 '경의중앙선',
 '경춘선',
 '공항철도선',
 '수인분당선',
 '신림선',
 '신분당선',
 '우이신설선'}

In [9]:
# 파일에 저장
station.to_csv("result_train_station.csv")